<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 2
<br>
<b>Deadline:</b> March 9, 2020 (Monday). 23:00
</div>

# Exercise 2.1. Backpropagation

In this assignment, you will implement forward and backward computations in a multilayer perceptron (MLP) network using pure [`numpy`](https://numpy.org). This will help you understand:
* The concept of backpropagation.
* How to implement and test gradient computations.
* How automatic differentiation is implemented in packages like [`PyTorch`](https://pytorch.org/docs/stable/index.html).

**In this notebook, we will implement MLP blocks that process only a single training sample at a time.**

**Foreward:**
* Please avoid using global variables within the functions.
* You need to brush up your knowledge of [`numpy`](https://numpy.org) to do this assignment.
* Inserting cells does not cause problems to autograding. You can insert cells to test your implementation of the gradient computations.

In [21]:
import numpy as np

First, we illustrate gradient computations with a simple example. Consider a function:

$$f(\mathbf{x}) = \mathbf{x}^T \mathbf{x} = x_1^2 + x_2^2.$$

Recall that gradient is a vector containing all partial derivatives of function $f$ w.r.t. all inputs
($x_1$ and $x_2$ in our case). We want to compute the gradient of $f(\mathbf{x})$ for any given input
$\mathbf{x}$.

The value of the gradient computed analytically is:
$$\nabla f(\mathbf{x}) = 2 \mathbf{x}$$
which for some (aribitrarily selected) input $\mathbf{x} = \begin{bmatrix}1\\2\end{bmatrix}$ is
$$\nabla f(\mathbf{x}) = 2 \mathbf{x} = \begin{bmatrix}2\\4\end{bmatrix}.$$
In this notebook, we call a vector computed in this way an *analytical gradient*.

To test our impementation of gradient computations, we can compute the value of the gradient for 
the same input $\mathbf{x} = \begin{bmatrix}1\\2\end{bmatrix}$ using [numerical differentiation](https://en.wikipedia.org/wiki/Numerical_differentiation):
$$ \nabla f(\mathbf{x}) \approx \frac{f(\mathbf{x} + \epsilon) - f(\mathbf{x} - \epsilon)}{2\epsilon}$$
which, using $\epsilon=0.5$, yields
$$\nabla f(\mathbf{x}) =
\frac{1}{2 \epsilon}
\begin{bmatrix}
  (1+\epsilon)^2 + 2^2 - ((1-\epsilon)^2 + 2^2)
\\
  1^2 + (2+\epsilon)^2 - (1^2 + (2-\epsilon)^2)
\end{bmatrix}
= \begin{bmatrix}1.9999\\4.0000\end{bmatrix}
$$
Note that the values of the numerical and analytical gradients  might _not exactly_ match to the decimal.

The function below implements numerical computations of the gradient.

In [22]:
def numerical_gradient(fun, x, eps=1e-4):
    """Compute derivatives of a given function fun numerically.
    
    Args:
      fun: A python function fun(x) which accepts a vector argument (one-dimensional numpy array)
           and returns a vector output (one-dimensional numpy array).
      x:   An input vector for which the numerical gradient should be computed.
      eps: A scalar which defines the magnitude of perturbations applied to the inputs
           (epsilon in the formula in the previous cell).

    Returns:
      gnum: A two-dimensional array in which an element in row i and column j is the partial derivative of the
            i-th output of function fun wrt j-th input of function fun (computed numerically).
    """
    assert x.ndim <= 1, "Only vector inputs are supported"
    e = np.zeros_like(x)
    f = fun(x)
    assert f.ndim <= 1, "Only vector outputs are supported"
    gnum = np.zeros((f.size, x.size))
    for i in range(len(x)):
        e[:] = 0
        e[i] = 1
        f1, f2 = fun(x + e*eps), fun(x - e * eps)
        gnum[:, i] = (f1 - f2) / (2 * eps)
    return gnum

<a id='reshaping_trick'></a>

**Reshaping trick**

Note that function `numerical_gradient` defined above accepts functions `fun` that works only with *one-dimensional arrays* as inputs and outputs.

Suppose we have function `fun(X)` which accepts a two-dimensional array `X` of shape `(n1, n2)` as input and produces a one-dimensional array `y` of shape `(ny,)` as output. We want to compute partial derivatives
`d y[i] / d X[k,l]` for each output element `y[i]` and each element `X[k,l]` of the input matrix. We can to it in the following way.

First, we define a function with one-dimensional inputs such that it can be passed to our `numerical_gradient`
function. Function `fun2` reshapes a one-dimensional array passed to it and calls function `fun`:
```
fun2 = lambda A: fun(A.reshape(n1, n2))
```

Then we can call the `numerical_gradient` function:
```
A = np.random.randn(n1, n2)
dA = numerical_gradient(fun2, A.flatten())
```
which will produce a two dimensional array of shape `(ny, n1*n2)` that will contain the required partial
derivatives.


## 1. Gradient of the loss

Let us first compute the gradient of a simple loss function wrt to its inputs. Consider the mean-squared error loss:
$$
c = \frac{1}{n} \sum_{i=1}^n (y_i - t_i)^2
$$
where $y_i$ are the elements of an input vector $\mathbf{y}$ and $t_i$ are the elements of the target vector $\mathbf{t}$.

In the code below, we define a class that performs forward and backward computations of this loss function. The `backward` function should compute the gradient $\frac{\partial c}{\partial \mathbf{y}}$.

In [23]:
class MSELoss:
    def forward(self, y, target):
        """
        Args:
          y (array):      Inputs of the loss function (can be, e.g., an output of a neural network),
                           shape (n_features,).
          target (array): Targets, shape (n_features,).
        """
        self.diff = diff = y - target  # Keep this for backward computations
        c = np.sum(np.square(diff)) / diff.size
        return c

    def backward(self):
        """
        Returns:
          dy (array): Gradient of the MSE loss wrt the inputs, shape (n_features,).
        """
        assert hasattr(self, 'diff'), "Need to call forward() first"

        dy=self.diff.__mul__((2/self.diff.size))
        return dy
        # YOUR CODE HERE
        # raise NotImplementedError()


In [24]:
def test_MSELoss_shapes():
    y = np.random.randn(3)
    target = np.zeros(3)  # Dummy target
    loss = MSELoss()  # Create the loss
    loss_value = loss.forward(y, target)  # Do forward computations
    dy = loss.backward()  # Do backward computations
    assert dy.shape == y.shape, f"Bad dy.shape: {dy.shape}"
    print('Success')

test_MSELoss_shapes()

Success


In [25]:
# Next we compare our implementation using numerical computations of the gradient
def test_MSELoss_backward():
    y = np.random.randn(3)
    target = np.zeros(3)  # Dummy target
    loss = MSELoss()  # Create the loss
    loss_value = loss.forward(y, target)  # Do forward computations

    dy = loss.backward()
    print('Analytical gradient:\n', dy)
    dy_num = numerical_gradient(lambda y: loss.forward(y, target), y)
    print('Numerical gradient:\n', dy_num[0])
    assert np.allclose(dy, dy_num), 'Analytical and numerical results differ'
    print('Success')

test_MSELoss_backward()

Analytical gradient:
 [ 0.65961179 -0.02219411  0.58831347]
Numerical gradient:
 [ 0.65961179 -0.02219411  0.58831347]
Success


If the input `y` of the loss function is the output of a neural network, now we know how to compute the gradient of the loss wrt the network's outputs.

## 2. Backpropagation through activation functions

Next we propagate the gradients further (backward) through the network. Suppose that somewhere in the network, we have *element-wise* nonlinearities applied to input vector $\mathbf{x}$ to produce outputs $\mathbf{y}$:
$$
\mathbf{y} = f(\mathbf{x}) \quad \text{such that} \quad y_i = f(x_i).
$$

When we backpropagate through that block, we need to transform the gradients $\frac{\partial c}{\partial \mathbf{y}}$ wrt to the outputs into the gradients wrt the inputs $\frac{\partial c}{\partial \mathbf{x}}$. Your task is to implement the forward and backward computations for `Tanh` nonlinearity.

Notes:
* We recommend you to compare analytical and numerical computations of the gradient.
* If you use function `numerical_gradient` to differentiate numerically `Tanh.forward()` using a one-dimensional array `x` as input, the output of `numerical_gradient` is a two-dimensional array (Jacobian matrix). We are interested only in the diagonal elements of that array because the nonlinearity is applied *element-wise*.

In [26]:
class Tanh:
    def forward(self, x):
        """
        Args:
          x (array): Input of shape (n_features,).
        
        Returns:
          y (array): Output of shape (n_features,).
        """
        y=np.tanh(x)
        self.x=x
        return y
        
        # YOUR CODE HERE
        # raise NotImplementedError()

    def backward(self, dy):
        """
        Args:
          dy (array): Gradient of a loss wrt outputs, shape (n_features,).
        
        Returns:
          dx (array): Gradient of a loss wrt inputs, shape (n_features,).
        """
        
        assert hasattr(self, 'x'), "Need to call forward() first."
        dx=dy/np.square(np.cosh(self.x))
        return dx
        
        # YOUR CODE HERE
        #raise NotImplementedError()

In [27]:
def test_Tanh_shapes():
    x = np.random.randn(3)
    act_fn = Tanh()
    y = act_fn.forward(x)
    dy = np.arange(1, 4)
    dx = act_fn.backward(dy)
    assert dx.shape == x.shape, f"Bad dx.shape: {dx.shape}"
    print('Success')

test_Tanh_shapes()

Success


We recommended you to compare analytical and numerical computations of the gradient.

In [28]:
# This cell tests Tanh

## 3. Backpropagation through a linear layer

Next we propagate the gradients (backward) through a linear layer. The linear layer implements forward computations:
$$
\mathbf{y} = \mathbf{W} \mathbf{x} + \mathbf{b}.
$$

In the backward pass, the linear layer receives the gradients wrt to the outputs $\frac{\partial c}{\partial \mathbf{y}}$ and it needs to compute:
* the gradients wrt the layer parameters $\mathbf{W}$ and $\mathbf{b}$
* the gradient $\frac{\partial c}{\partial \mathbf{x}}$ wrt the inputs.

We implement the forward and the backward computations in the cell below.

In [29]:
def linear_forward(x, W, b):
    """Forward computations in a linear layer:
        y = W x + b

    Args:
      x (array): Input of shape (in_features,).
      W (array): Weight matrix of shape (out_features, in_features).
      b (array): Bias term of shape (out_features,).

    Returns:
      y (array): Output of shape (out_features,).
    """
    return np.dot(W, x) + b

def linear_backward(dy, x, W, b):
    """Backward computations in a linear layer.

    Args:
      dy (array): Gradient of a loss wrt outputs, shape (out_features,).
      x (array): Input of shape (in_features,).
      W (array): Weight matrix of shape (out_features, in_features).
      b (array): Bias term of shape (out_features,).

    Returns:
      dx (array): Gradient wrt inputs (in_features,).
      dW (array): Gradient wrt weight matrix W, shape (out_features, in_features).
      db (array): Gradient wrt bias term b, shape (out_features,).
    """
    dx=dy@W
  
    dW=np.outer(dy, x)
    db=dy
    
    return dx, dW, db
    # YOUR CODE HERE
    # raise NotImplementedError()

In [30]:
def test_linear_shapes():
    x = np.random.randn(2)
    W = np.random.randn(3, 2)
    b = np.random.randn(3)

    y = linear_forward(x, W, b)
    dy = np.arange(1, 4)  # Gradient wrt y selected arbitrarily
    dx, dW, db = linear_backward(dy, x, W, b)
    assert dx.shape == x.shape, f"Bad dx.shape: {dx.shape}"
    assert dW.shape == W.shape, f"Bad dW.shape: {dW.shape}"
    assert db.shape == b.shape, f"Bad db.shape: {db.shape}"
    print('Success')

test_linear_shapes()

Success


Now we test the computations of $\frac{\partial c}{\partial \mathbf{W}}$ numerically.

In the code below, we used the [reshaping trick](#reshaping_trick) because we want to compute derivatives
wrt all elements of a two-dimensional input `W` of function `linear_forward`. Function `numerical_gradient`
returns a two-dimensional array of shape `(3, 3*2)` which we combine further with the selected value of `dy`.

In [11]:
# Test gradient wrt W numerically
def test_dW_numerically():
    x = np.random.randn(2)
    W = np.random.randn(3, 2)
    b = np.random.randn(3)

    y = linear_forward(x, W, b)
    dy = np.arange(1, 4)  # Gradient wrt y selected arbitrarily
    dx, dW, db = linear_backward(dy, x, W, b)
    print('Analytical gradient:\n', dW)

    dW_num = numerical_gradient(lambda W: linear_forward(x, W.reshape(3, 2), b), W.flatten())
    dW_num = np.dot(dy.T, dW_num).reshape(3, 2)
    print('Numerical gradient:\n', dW_num)
    assert np.allclose(dW, dW_num), 'Analytical and numerical results differ.'
    print('Success')

test_dW_numerically()

Analytical gradient:
 [[ 0.49043575 -0.20452946]
 [ 0.98087151 -0.40905892]
 [ 1.47130726 -0.61358838]]
Numerical gradient:
 [[ 0.49043575 -0.20452946]
 [ 0.98087151 -0.40905892]
 [ 1.47130726 -0.61358838]]
Success


We recommend you to compare analytical and numerical computations of the gradients also wrt input `x` and bias term `b`.

In [12]:
# This cell tests linear_forward and linear_backward

In [13]:
# This cell tests linear_forward and linear_backward

In [14]:
# This cell tests linear_forward and linear_backward

Next we implement a class that represents a linear layer.

In [15]:
class Linear:
    def __init__(self, in_features, out_features):
        """
        Args:
          in_features (int): Number of input features.
          out_features (int): Number of output features.
        """
        self.W = np.random.randn(out_features, in_features)
        self.b = np.random.randn(out_features)

        self.grad_W = None
        self.grad_b = None

    def forward(self, x):
        """
        Args:
          x (array): Input of shape (in_features,).
        
        Returns:
          y (array): Output of shape (out_features,).
        """
        self.x = x  # Keep this for backward computations
        return linear_forward(x, self.W, self.b)

    def backward(self, dy):
        """
        Args:
          dy (array): Gradient of a loss wrt outputs, shape (out_features,).
        
        Returns:
          dx (array): Gradient of a loss wrt inputs, shape (in_features,).
        """
        assert hasattr(self, 'x'), "Need to call forward() first."
        assert dy.size == self.W.shape[0]
        dx, self.grad_W, self.grad_b = linear_backward(dy, self.x, self.W, self.b)
        return dx

In [16]:
# We can now create a linear layer, ...
layer = Linear(in_features=3, out_features=2)

# do forward computations ...
x = np.random.randn(3)
y = layer.forward(x)

# and backward computations
dy = np.arange(1, 3)
dx = layer.backward(dy)

# We now have the gradients computed
# wrt input x
assert dx.shape == x.shape, f"Bad dx.shape: {dx.shape}"
# wrt weight matrix W
assert layer.grad_W.shape == layer.W.shape, f"Bad grad_W.shape: {layer.grad_W.shape}, W.shape={layer.W.shape}"
# wrt bias term b
assert layer.grad_b.shape == layer.b.shape, f"Bad grad_b.shape: {layer.grad_b.shape}, b.shape={layer.b.shape}"

## 4. Define a multilayer perceptron and do full backpropagation

Now let us define a multilayer perceptron and do backpropagation of the gradients from its outputs to its inputs. Your task is to implement forward and backward computations.

In [17]:
class MLP:
    def __init__(self, in_features, hidden_size1, hidden_size2, out_features):
        """Multilayer perceptron network with two hidden layers and tanh nonlinearity in both hidden layers.
        
        Args:
          in_features (int): Number of inputs.
          hidden_size1 (int): Number of units in the first hidden layer.
          hidden_size2 (int): Number of units in the second hidden layer.
          out_features (int): Number of outputs.
        """
        self.fc1 = Linear(in_features, hidden_size1)
        self.activation_fn1 = Tanh()
        self.fc2 = Linear(hidden_size1, hidden_size2)
        self.activation_fn2 = Tanh()
        self.fc3 = Linear(hidden_size2, out_features)

    def forward(self, x):
        """
        Args:
          x (array): Input of shape (in_features,).
        
        Returns:
          y (array): Output of shape (out_features,).
        """
        # YOUR CODE HERE
        # raise NotImplementedError()
        
        self.x = x
        
        y = self.fc1.forward(x)
        y = self.activation_fn1.forward(y)
        y = self.fc2.forward(y)
        y = self.activation_fn2.forward(y)
        y = self.fc3.forward(y)
        
        return y

    def backward(self, dy):
        """
        Args:
          dy (array): Gradient of a loss wrt outputs, shape (out_features,).
        
        Returns:
          dx (array): Gradient of a loss wrt inputs, shape (in_features,).
        """
        # YOUR CODE HERE
        # raise NotImplementedError()
        dx=self.fc3.backward(dy)
        dx=self.activation_fn2.backward(dx)
        dx=self.fc2.backward(dx)
        dx=self.activation_fn1.backward(dx)
        dx=self.fc1.backward(dx)
        
        return dx

In [18]:
def test_MLP():
    x = np.random.randn(2)
    mlp = MLP(2, 10, 20, 5)
    y = mlp.forward(x)
    dy = np.arange(1, 6)  # gradient of a loss function wrt MLP's outputs
    dx = mlp.backward(dy)

    assert dx.shape == x.shape, f"Bad dx.shape: {dx.shape}"

    # Test gradient wrt x numerically
    print('Analytical gradient:\n', dx)

    dx_num = numerical_gradient(lambda x: mlp.forward(x), x)
    dx_num = np.dot(dy.T, dx_num)
    print('Numerical gradient:\n', dx_num)
    assert np.allclose(dx, dx_num), 'Analytical and numerical results differ'
    print('Success')

test_MLP()

Analytical gradient:
 [31.78842458 -8.78987605]
Numerical gradient:
 [31.78842466 -8.78987607]
Success


In [19]:
# This cell tests MLP

Now we can connect the output of the MLP to the `MSELoss` defined previously and do full backpropagation.

In [20]:
mlp = MLP(2, 10, 20, 5)
loss = MSELoss()

# Dummy inputs and targets
x = np.random.randn(2)
target = np.random.randn(5)

# Forward computations
y = mlp.forward(x)
assert y.shape == target.shape, f"Bad y.shape: {y.shape}"
c = loss.forward(y, target)

# Backward computations
dy = loss.backward()
assert dy.shape == y.shape, f"Bad dy.shape={dy.shape}"
mlp.backward(dy)

# To update the parameters of the MLP, we can use the gradients, for example:
print('The gradient wrt the weights of the first layer:\n', mlp.fc1.grad_W)

The gradient wrt the weights of the first layer:
 [[-0.32952488  0.84472992]
 [-1.03219875  2.64601921]
 [-1.26668651  3.24712352]
 [-1.07098929  2.74545792]
 [ 1.98285325 -5.0830015 ]
 [ 0.3235166  -0.82932782]
 [ 0.29051013 -0.74471645]
 [-0.32366491  0.829708  ]
 [ 0.36676996 -0.94020687]
 [ 0.00562815 -0.01442765]]


<div class="alert alert-block alert-info">
<b>Conclusions</b>
</div>

Now you have an idea how automatic differentiation is implemented in packages like PyTorch.

Note that the code above works only for propagating the error of a single data point (not a set of training examples).